In [1]:
# Import basic libraries
from palmerpenguins import load_penguins
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import time
from sklearn.preprocessing import LabelEncoder
import os
os.chdir("/Users/hoangthuyduongvu/Desktop/FuzzSIM")


In [2]:
# Import personalized libraries
from fuzz.src.capacity import *
from fuzz.choquet.choquet import *
from fuzz.src.norm import *
from fuzz.src.knn import KNNFuzz
from fuzz.src.sim import S1, S2, S3
from fuzz.optim import *
from fuzz.utils import *
from fuzz.eval import leave_one_out
from fuzz.dataloader import *
from fuzz.choquet.d_choquet import *

# BASIC

In [3]:
# Load data 
iris_data = load_iris()
iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
iris['target'] = iris_data.target
data, labels = iris.iloc[:, :-1].values, iris.iloc[:, -1].values
data = batch_norm(np.array(data, dtype=np.float32))

In [7]:
print(monotonic_check_unit(
    X = Capacity(X=[1,2,3], mu=0.4),
    Y = Capacity(X=[2,3], mu=0.1)
))  # -> True
print(monotonic_check_unit(
    X = Capacity(X=[1,2,3], mu=0.04),
    Y = Capacity(X=[2,3], mu=0.1)
))  # -> False
print(monotonic_check_unit(
    X = Capacity(X=[1,3], mu=0.04),
    Y = Capacity(X=[2,3], mu=0.1)
))  # -> True

True
False
True


In [12]:
mu=generate_capacity(enumerate_permute_unit(data[0]))
for i in range(len(mu)):
    print(f"Capaicity of {mu[i].X} is {mu[i].mu}")
print()
print("Monotonicity check:", monotonic_check(mu))   # -> True

Capaicity of [] is 0
Capaicity of [3] is 0.027537024529366128
Capaicity of [2] is 0.10154185134699456
Capaicity of [1] is 0.10709754140919547
Capaicity of [0] is 0.17624043600849904
Capaicity of [3, 2] is 0.1894521068673024
Capaicity of [3, 1] is 0.19809721233273314
Capaicity of [3, 0] is 0.36116704633526087
Capaicity of [2, 1] is 0.4307516159361944
Capaicity of [2, 0] is 0.5391636122574057
Capaicity of [1, 0] is 0.6572586003972716
Capaicity of [3, 2, 1] is 0.7080812490800965
Capaicity of [3, 2, 0] is 0.9238727546009731
Capaicity of [3, 1, 0] is 0.9355529448509242
Capaicity of [2, 1, 0] is 0.9675076441460724
Capaicity of [3, 2, 1, 0] is 1

Monotonicity check: True


In [13]:
features = data[0]

mobius = generate_mobius(features, 2)
capacity = mobius_to_capacity(mobius, features)

print("Möbius:")
for i in range(len(mobius)):
    print(f"Mobius of {mobius[i].X} is {mobius[i].mu:.3f}")

print("\nDerived Capacity:")
for c in capacity:
    print(f"Capacity of {c.X} is {c.mu:.3f}")


Möbius:
Mobius of [] is 0.000
Mobius of [np.float64(1.0)] is 0.898
Mobius of [np.float64(0.6734693646430969)] is 0.163
Mobius of [np.float64(0.24489794671535492)] is 0.144
Mobius of [np.float64(0.0)] is 0.589
Mobius of [np.float64(0.6734693646430969), np.float64(1.0)] is 0.665
Mobius of [np.float64(0.24489794671535492), np.float64(1.0)] is 0.651
Mobius of [np.float64(0.0), np.float64(1.0)] is 0.289
Mobius of [np.float64(0.6734693646430969), np.float64(0.24489794671535492)] is 0.969
Mobius of [np.float64(0.6734693646430969), np.float64(0.0)] is 0.969
Mobius of [np.float64(0.24489794671535492), np.float64(0.0)] is 0.329

Derived Capacity:
Capacity of [] is 0.000
Capacity of [np.float64(1.0)] is 0.158
Capacity of [np.float64(0.6734693646430969)] is 0.029
Capacity of [np.float64(0.24489794671535492)] is 0.025
Capacity of [np.float64(0.0)] is 0.104
Capacity of [np.float64(0.6734693646430969), np.float64(1.0)] is 0.305
Capacity of [np.float64(0.24489794671535492), np.float64(1.0)] is 0.299
C